#### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT1. Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
# One VS Rest 구현을 위해 1) class가 0 or not 2)class가 1 or not을 구분하기 위한 머신 등을 미리 만들어주자
svm_1 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_2 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_3 = SVC(kernel ='rbf', C = 5, gamma = 5)

In [4]:
from sklearn.model_selection import train_test_split #test/train 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
y_train = pd.get_dummies(y_train) #one hot encoding

In [7]:
y_train

,setosa,versicolor,virginica
110,0,0,1
69,0,1,0
148,0,0,1
39,1,0,0
53,0,1,0
...,...,...,...
64,0,1,0
91,0,1,0
81,0,1,0
51,0,1,0


In [8]:
svm_1.fit(X_train,y_train.iloc[:,0]) # 데이터 클레스가 0 or not 구분해주는 머신
svm_2.fit(X_train,y_train.iloc[:,1]) # 데이터 클레스가 1 or not 구분해주는 머신
svm_3.fit(X_train,y_train.iloc[:,2]) # 데이터 클레스가 2 or not 구분해주는 머신
print(svm_1.predict(X_test)) #데이터 클래스가 0 or not을 구분해주는 애를 통해서 테스트 데이터의 클래스가 0인지, 0이 아닌인지 예측해보자

print(svm_1.decision_function(X_test)) #decision_function hyperplane과의 거리를 구하는 방법(필요하다면 활용해주세요!)

[0 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0]
[-1.12470845 -0.86326953 -0.65281099 -0.50248821 -0.76284323 -0.87465573
  1.07709345 -0.99281647  0.47441336 -0.99842743 -0.83989348  0.15633457
  0.32871788 -0.97965464 -0.72385083 -0.92638376  1.28322481 -0.56240455
 -0.72719892 -0.99509775  0.43166724 -0.96451557 -0.82991366 -1.03020581
 -0.75166835  1.13461335  0.39943974 -1.04194106 -0.93376548 -1.06133798]


In [9]:
# 부호가 모든 같은 경우가 있는가? > 모두 동점인 경우가 생길 것
for i in range(len(X_test)):
    # ~. decision_function을 이용하면 해당 데이터가 하이퍼플레인으로부터 얼마나 떨어져있는지 '거리'가 나온다!
    # 다음은 그 값의 부호를 이용해 모두가 동점인 경우가 있는지 출력하는 함수 
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) \
    and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
        print(i)

3
17
18


In [10]:
# 동점인 경우를 모은 list 생성
tie_case = []
for i in range(len(X_test)):
    # ~. decision_function을 이용하면 해당 데이터가 하이퍼플레인으로부터 얼마나 떨어져있는지 '거리'가 나온다!
    # 다음은 그 값의 부호를 이용해 모두가 동점인 경우가 있는지 출력하는 함수 
    if (np.sign(svm_1.decision_function(X_test)[i]) == np.sign(svm_2.decision_function(X_test)[i])) \
    and (np.sign(svm_2.decision_function(X_test)[i]) == np.sign(svm_3.decision_function(X_test)[i])):
        tie_case.append(i)
        
tie_case

[3, 17, 18]

In [12]:
## Case 1 : One vs Rest SVM을 이부분에 구현해주세요 위 결과들을 이용해서 multi class SVM을 직접 구현해주세요! 하드코딩이 하시기 편할겁니다.
import random 
from sklearn import metrics

# 각 svm 이진분류기가 예측한 값을 dataframe 형식으로 저장
df = pd.DataFrame()
df['setosa'] = svm_1.predict(X_test)
df['versicolor'] = svm_2.predict(X_test)
df['virginica'] = svm_3.predict(X_test)

# 각 예측값 중 1에 해당하는 값을 가진 label을 Label list에 저장
Label = []
for i in range(len(df)):
    # 만약 동점인 상황일 경우, decision_function 값이 가장 큰 label을 저장하기로 함 
    if i not in tie_case:
        Label.append(np.argmax(df.iloc[i,:]))
    else:
        if np.argmax([svm_1.decision_function(X_test)[i],svm_2.decision_function(X_test)[i],svm_3.decision_function(X_test)[i]])==0:
                      Label.append('setosa')
        elif np.argmax([svm_1.decision_function(X_test)[i],svm_2.decision_function(X_test)[i],svm_3.decision_function(X_test)[i]])==1:
                        Label.append('versicolor')
        else:
                        Label.append('virginica')
                      
    
print(Label)
print(metrics.accuracy_score(y_test, Label))

['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']
0.8666666666666667


C:\Users\jhr50\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)
C:\Users\jhr50\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)
C:\Users\jhr50\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behav

In [ ]:
# Case 2 : One vs One SVM을 이 부분에 구현해주세요. (선택사항)


['versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica', 'virginica', 'setosa', 'virginica', 'setosa', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'versicolor', 'versicolor', 'setosa', 'versicolor', 'virginica', 'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa', 'virginica', 'virginica', 'versicolor']


0.8666666666666667

In [18]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
from sklearn.model_selection import train_test_split #데이터셋 분리
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=48)

svm_4 = SVC(kernel ='rbf', C = 5, gamma = 5)
svm_4.fit(X_train_2, y_train_2)
y_pred = svm_4.predict(X_test_2)

metrics.accuracy_score(y_test_2,y_pred)

0.8333333333333334